## Cover page placeholder

In [ ]:
# pip install python-constraint

# AI - Hiring Problem (CSP) 

## Scenario 1 
### Ciara knows Python, and only has funds to hire three more people.

It won't be possible to fill all of positions with just 3 hires, so Ciara will have to take on one of the Python positions.

In [74]:
from constraint import Problem

# Defines the positions and the hires with their qualifications
positions = ["Python_1", "Python_2", "AI_1", "AI_2", "Web", "Database", "Systems"]
hires = {
    "Ciara": ["Python_1"],
    "Peter": ["Python_2", "AI_1", "AI_2"],
    "Juan": ["Web", "AI_1", "AI_2"],
    "Jim": ["AI_1", "AI_2", "Systems"],
    "Jane": ["Python_2", "Database"],
    "Mary": ["Web", "Systems"],
    "Bruce": ["Systems", "Python_2"],
    "Anita": ["Web", "AI_1", "AI_2"]
}

problem = Problem()

# Defines the variables (positions) and their domains (hires who can take the positions)
for position in positions:
    problem.addVariable(position, list(hires.keys()))

# Defines the constraints
'''
IMPORTANT: Ciara only has the funds to hire 3 more people
She knows python and will fill one of the python positions
for this reason
'''
# Ciara will be taking a python position
def ciara_python_constraint(python_1):
    return python_1 == "Ciara"

# A person can only ocuppy one of the two available AI positions
def ai_constraint(ai_1, ai_2):
    return ai_1 != ai_2

# Jane is the only one qualified to ocuppy the DB position 
def db_constraint(database):
    return database == "Jane"

# Defines the amount of people that will be hired (including Ciara)
def ciara_hiring_constraint(*positions):
    return len(set(positions)) <= 4

# Defines that a person can only occupy a max of two positions
def max_two_positions_constraint(*positions):
    hires_count = dict()
    for hire in positions:
        if hire in hires_count:
            hires_count[hire] += 1
        else:
            hires_count[hire] = 1
    return all(count <= 2 for count in hires_count.values())

# Defines the constraint that an individual can only occupy the positions they are qualified for
def qualification_constraint(*args):
    for position, hire in zip(positions, args):
        if position not in hires[hire]:
            return False
    return True

# Adds the constraints to the problem
problem.addConstraint(ciara_python_constraint, ["Python_1"])
problem.addConstraint(ai_constraint, ["AI_1", "AI_2"])
problem.addConstraint(db_constraint, ["Database"])
problem.addConstraint(ciara_hiring_constraint, positions)
problem.addConstraint(max_two_positions_constraint, positions)
problem.addConstraint(qualification_constraint, positions)

# Solutions
solutions = problem.getSolutions()

# Removes duplicates
unique_solutions1 = []
for solution in solutions:
    # For positions with the same qualifications, sort the individuals
    ai_positions = sorted([solution["AI_1"], solution["AI_2"]])
    solution["AI_1"], solution["AI_2"] = ai_positions[0], ai_positions[1]
    
    # Adds to unique solutions if not present already
    if solution not in unique_solutions1:
        unique_solutions1.append(solution)

# Prints the solutions by position
for i, solution in enumerate(unique_solutions1):
    print(f"Solution {i+1}:")
    for position, hire in solution.items():
        print(f"Position {position}: {hire}")
    print()

Solution 1:
Position AI_1: Anita
Position AI_2: Jim
Position Database: Jane
Position Python_1: Ciara
Position Python_2: Jane
Position Systems: Jim
Position Web: Anita

Solution 2:
Position AI_1: Jim
Position AI_2: Juan
Position Database: Jane
Position Python_1: Ciara
Position Python_2: Jane
Position Systems: Jim
Position Web: Juan



## Scenario 2
### Ciara and Juan become partners, with the additional funds they can now employ four more people but must employ another AI Engineer, so they need 2 Python Programmers, 3 AI Engineers, 1 Web Designer, 1 Database Admin, and 1 Systems Engineer.

This scenario will assume the following hiring conditions:

- Ciara and Juan will be considered for the positions they are qualified for.
- Solutions with Ciara OR Juan OR none of them are also going to be considered.

In [75]:
from constraint import Problem

# Defines the positions and the hires with their qualifications
positions = ["Python_1", "Python_2", "AI_1", "AI_2", "AI_3", "Web", "Database", "Systems"]
hires = {
    "Ciara": ["Python_1", "Python_2"],
    "Peter": ["Python_1", "Python_2", "AI_1", "AI_2", "AI_3"],
    "Juan": ["Web", "AI_1", "AI_2", "AI_3"],
    "Jim": ["AI_1", "AI_2", "AI_3", "Systems"],
    "Jane": ["Python_1", "Python_2", "Database"],
    "Mary": ["Web", "Systems"],
    "Bruce": ["Systems", "Python_1", "Python_2"],
    "Anita": ["Web", "AI_1", "AI_2", "AI_3"]
}

problem = Problem()

# Defines the variables (positions) and their domains (hires who can take the positions)
for position in positions:
    problem.addVariable(position, list(hires.keys()))

# # Defines the constraints
def python_constraint(python_1, python_2):
    return python_1 != python_2

def ai_constraint(ai_1, ai_2, ai_3):
    return ai_1 != ai_2 and ai_1 != ai_3 and ai_2 != ai_3

# Jane is the only one qualified to occupy the DB position 
def db_constraint(database):
    return database == "Jane"

# Defines the amount of people that can be hired (Including Ciara and Juan)
def hiring_constraint(*args):
    hires_set = set(args)
    if "Ciara" in hires_set and "Juan" in hires_set:
        return len(hires_set) <= 6
    elif "Ciara" in hires_set or "Juan" in hires_set:
        return len(hires_set) <= 5
    else:
        return len(hires_set) == 4

# Defines that a person can only occupy a max of two positions
def max_two_positions_constraint(*args):
    hires_count = dict()
    for hire in args:
        if hire in hires_count:
            hires_count[hire] += 1
        else:
            hires_count[hire] = 1
    return all(count <= 2 for count in hires_count.values())

# Defines the constraint that an individual can only occupy the positions they are qualified for
def qualification_constraint(*args):
    for position, hire in zip(positions, args):
        if position not in hires[hire]:
            return False
    return True

# Adds the constraints to the problem
problem.addConstraint(python_constraint, ["Python_1", "Python_2"])
problem.addConstraint(ai_constraint, ["AI_1", "AI_2", "AI_3"])
problem.addConstraint(db_constraint, ["Database"])
problem.addConstraint(hiring_constraint, positions)
problem.addConstraint(max_two_positions_constraint, positions)
problem.addConstraint(qualification_constraint, positions)

# Gets and prints solutions
solutions = problem.getSolutions()

total_combinations = 0
unique_solutions2 = []
for solution in solutions:
    # For positions with the same qualifications, sorts the individuals
    ai_positions = sorted([solution["AI_1"], solution["AI_2"], solution["AI_3"]])
    solution["AI_1"], solution["AI_2"], solution["AI_3"] = ai_positions[0], ai_positions[1], ai_positions[2]

    python_positions = sorted([solution["Python_1"], solution["Python_2"]])
    solution["Python_1"], solution["Python_2"] = python_positions

    # Adds to unique solutions if not already present
    if solution not in unique_solutions2:
        unique_solutions2.append(solution)
        total_combinations+=1


# Prints the solutions by position
for i, solution in enumerate(unique_solutions2):   
    print(f"Solution {i+1}:")
    for position, hire in solution.items():
        print(f"Position {position}: {hire}")
    print()

Solution 1:
Position AI_1: Anita
Position AI_2: Jim
Position AI_3: Juan
Position Python_1: Bruce
Position Python_2: Ciara
Position Database: Jane
Position Systems: Bruce
Position Web: Juan

Solution 2:
Position AI_1: Anita
Position AI_2: Jim
Position AI_3: Juan
Position Python_1: Bruce
Position Python_2: Ciara
Position Database: Jane
Position Systems: Bruce
Position Web: Anita

Solution 3:
Position AI_1: Anita
Position AI_2: Jim
Position AI_3: Juan
Position Python_1: Bruce
Position Python_2: Ciara
Position Database: Jane
Position Systems: Jim
Position Web: Juan

Solution 4:
Position AI_1: Anita
Position AI_2: Jim
Position AI_3: Juan
Position Python_1: Bruce
Position Python_2: Ciara
Position Database: Jane
Position Systems: Jim
Position Web: Anita

Solution 5:
Position AI_1: Anita
Position AI_2: Jim
Position AI_3: Juan
Position Python_1: Bruce
Position Python_2: Jane
Position Database: Jane
Position Systems: Bruce
Position Web: Juan

Solution 6:
Position AI_1: Anita
Position AI_2: Jim
P

## Information about the hiring combinations for Scenario 2

In [141]:
# Initialises a dictionary to store the counts for each person
person_counts = dict()

# Loops over the unique solutions
for solution in unique_solutions2:
    # GetS the set of hires in each solution
    hires_set = set(solution.values())
    # LoopS over the hires in the set
    for hire in hires_set:
        # IncrementS the count for the hire by one
        person_counts[hire] = person_counts.get(hire, 0) + 1

# Defines a function to extract the count from an item
def get_count(item):
    return item[1]

# Sorts the person counts by ascending order of values
person_counts = dict(sorted(person_counts.items(), key=get_count))

print(f"There is a total of {total_combinations} hiring combinations")
print()
# Prints the counts for each person
for person, count in person_counts.items():
    if count >= total_combinations:
        print(f"{person} appears in ALL solutions")
    else:
        print(f"{person} appears in {count} solutions.")

print()

# Prints solutions with specific hires
solutions_without_ciara_count = 0
solutions_without_juan_count = 0
solutions_without_neither_count = 0
solutions_with_both_count = 0

for i, solution in enumerate(unique_solutions2):
    if "Ciara" not in solution.values():
        solutions_without_ciara_count += 1
    if "Juan" not in solution.values():
        solutions_without_juan_count += 1
    if "Ciara" not in solution.values() and "Juan" not in solution.values():
        solutions_without_neither_count += 1
    if "Ciara" in solution.values() and "Juan" in solution.values():
        solutions_with_both_count += 1  
        
print(f"The number of solutions that Ciara is not in is: {solutions_without_ciara_count}")
print(f"The number of solutions that Juan is not in is: {solutions_without_juan_count}")
print(f"The number of solutions that neither is in is: {solutions_without_neither_count}")
print(f"The number of solutions with BOTH in it is: {solutions_with_both_count}")

There is a total of 77 hiring combinations

Mary appears in 22 solutions.
Bruce appears in 31 solutions.
Ciara appears in 48 solutions.
Anita appears in 56 solutions.
Jim appears in 56 solutions.
Peter appears in 61 solutions.
Juan appears in 74 solutions.
Jane appears in ALL solutions

The number of solutions that Ciara is not in is: 29
The number of solutions that Juan is not in is: 3
The number of solutions that neither is in is: 1
The number of solutions with BOTH in it is: 46


## Solution where neither Ciara nor Juan are needed

In [77]:
for i, solution in enumerate(unique_solutions2):
    if "Ciara" not in solution.values() and "Juan" not in solution.values():
        print(f"Solution {i+1}:")
        for position, hire in solution.items():
            print(f"Position {position}: {hire}")
        print()     

Solution 24:
Position AI_1: Anita
Position AI_2: Jim
Position AI_3: Peter
Position Python_1: Jane
Position Python_2: Peter
Position Database: Jane
Position Systems: Jim
Position Web: Anita



# Exercise 3: Solving with a different algorithm

## Scenario 1 without CSP

Backtracking + DFS

In [78]:
# Initialises a dictionary to store the positions and each hire that can fill it
positions = {
    "Python_1": ["Ciara"],
    "Python_2": ["Peter", "Jane", "Bruce"],
    "AI_1": ["Peter", "Juan", "Jim", "Anita"],
    "AI_2": ["Peter", "Juan", "Jim", "Anita"],
    "Web": ["Juan", "Mary", "Anita"],
    "Database": ["Jane"],
    "Systems": ["Jim", "Mary", "Bruce"]
}

# Initialises an empty set to store the seen assignments
seen = set()

def assign(positions, hires, assignment={}, unique_hires=4):
    # If all positions are filled, returns the assignment
    if len(assignment) == len(positions):
        # Checks if AI_1 and AI_2 hires are different
        if assignment.get("AI_1") != assignment.get("AI_2"): 
            # Creates a tuple of the hires assigned to AI_1 and AI_2 and sorts it in alphabetical order
            ai_hires = tuple(sorted([assignment["AI_1"], assignment["AI_2"]]))
            # Checks if the tuple is already in the seen set
            if ai_hires not in seen:
                # Adds the tuple to the seen set
                seen.add(ai_hires)
                # Returns the assignment
                yield assignment
    else:
        # Gets the next position to fill
        position = next(pos for pos in positions if pos not in assignment)
        # Tries to assign each hire to the position
        for hire in positions[position]:
            # Checks if the hire is already assigned too many times or goes above the unique_hires
            if list(assignment.values()).count(hire) < 2 and len(set(list(assignment.values()) + [hire])) <= unique_hires:
                # Assign the hire to the position
                assignment[position] = hire
                # Fills the remaining positions using recursion
                yield from assign(positions, hires, assignment)
                # Removes the assignment to backtrack
                del assignment[position]       

# Prints tbe possible solutions
for i, assignment in enumerate(assign(positions, positions.values()), 1):
    print(f"Solution {i}:")
    for position, hire in assignment.items():    
        print(f"Position {position}: {hire}")
    print()


Solution 1:
Position Python_1: Ciara
Position Python_2: Jane
Position AI_1: Juan
Position AI_2: Jim
Position Web: Juan
Position Database: Jane
Position Systems: Jim

Solution 2:
Position Python_1: Ciara
Position Python_2: Jane
Position AI_1: Jim
Position AI_2: Anita
Position Web: Anita
Position Database: Jane
Position Systems: Jim



# Data Visualization Tasks - Visualizations and GUI

## Scenario 1

In [193]:
from IPython.display import display, HTML

# Define the positions and the hires with their qualifications
positions1 = ["Python 1", "Python 2", "AI 1", 
             "AI 2", "Web", "Database", "Systems"]

hires_sc1 = {
    "Ciara": ["Python 1"],
    "Peter": ["Python 2", "AI 1", "AI 2"],
    "Juan": ["Web", "AI 1", "AI 2"],
    "Jim": ["AI 1", "AI 2", "Systems"],
    "Jane": ["Python 2", "Database"],
    "Mary": ["Web", "Systems"],
    "Bruce": ["Systems", "Python 2"],
    "Anita": ["Web", "AI 1", "AI 2"]
}

# Create a DataFrame with the available positions
df_positions1 = pd.DataFrame(positions1, columns=["Available Positions"])

# Convert the DataFrame to HTML
html_positions1 = df_positions1.to_html(index=False)

# DataFrame for each person, stored in a list
dfs1 = [pd.DataFrame({person: positions1}) for person, positions1 in hires_sc1.items()]

# Converts each DataFrame to HTML
html_strs = [df.to_html(index=False) for df in dfs1]

# Combine the HTML strings and display them
html_str = "<div style='text-align:center;'>" + \
            "<h2>Available Positions</h2>" + \
            "<div style='display:flex; flex-direction:row; justify-content:center;'>" + \
            f"<div style='padding:10px;'>{html_positions1}</div>" + \
            "</div></div>"

html_str += "<div style='text-align:center;'>" + \
           "<h2>Positions candidates are qualified to occupy</h2>" + \
           "<div style='display:flex; flex-direction:row; justify-content:center;'>" + \
           ''.join([f"<div style='padding:10px;'>{table}</div>" for table in html_strs]) + \
           "</div></div>"

display(HTML(html_str))

Available Positions
Python 1
Python 2
AI 1
AI 2
Web
Database
Systems
Ciara
Python 1
Peter


In [79]:
import pandas as pd

# Convert the list of solutions into a DataFrame
df_scenario1 = pd.DataFrame(unique_solutions1)

df_scenario1.index.name = "Hiring option"
df_scenario1.reset_index(inplace=True)
# Remove the index name
df_scenario1.set_index("Hiring option", inplace=True)

df_scenario1.columns.name = df_scenario1.index.name
df_scenario1.index.name = None
df_scenario1.index = range(1, len(df_scenario1) + 1)


df_scenario1

Hiring option,AI_1,AI_2,Database,Python_1,Python_2,Systems,Web
1,Anita,Jim,Jane,Ciara,Jane,Jim,Anita
2,Jim,Juan,Jane,Ciara,Jane,Jim,Juan


## Scenario 1 - Sankey Diagram

### Hiring Options Overview

In [80]:
import plotly.graph_objects as go

# Initialize lists to store the source, target, and value data for the Sankey diagram
sources = []
targets = []
values = []

# Iterate over the cells in the dataframe
for i, row in df_scenario1.iterrows():
    for j, value in row.items():  # Use row.items() instead of row.iteritems()
        # Add the data to the lists
        sources.append(i)
        targets.append(value)
        values.append(1)  # assuming each pair occurs only once

# Create a dictionary that maps each unique label to a unique index
labels = list(set(sources + targets))
label_to_index = {label: index for index, label in enumerate(labels)}

# Convert the sources and targets to indices
source_indices = [label_to_index[source] for source in sources]
target_indices = [label_to_index[target] for target in targets]

# Define the source-target pairs
link = dict(
  arrowlen=15,
  source = source_indices,
  target = target_indices,
  value = values
)

# Define the node labels
node = dict(
  label = labels,
  pad = 15,  # 15 pixel padding
  thickness = 20  # 20 pixel thickness
)

# Create the Sankey diagram object
data = go.Sankey(link = link, node = node)

# Create the layout
layout =  dict(title = "Hiring Options - Scenario 1 - Overview",
    font = dict(size = 10))

# Create the figure
fig = go.Figure(data=[data], layout=layout)

# Plot the figure
fig.show()

## Hiring options in Detail - Position and Candidate

In [81]:
# Defines the source-target pairs for hiring options
link = dict(
  arrowlen=15,
  source = [0, 0, 1, 1, 2, 3, 4], # Refers to the positions: Python(0,0), AI(1,1), Web(2), DB(3), Sys(4)
  target = [5, 6, 7, 8, 8, 6, 7], # Refers to the candidates: Ciara(5), Jane(6), Anita/Juan(7), Jim(8)
  value = [1, 1, 1, 1, 1, 1, 1] # Each pair occurs only once
)

# Defines the node labels for hiring option 1
node1 = dict(
  label = ["Python Programmer", "AI Engineer", "Web Designer","Database Administrator","Systems Engineer",
           "Ciara", "Jane", "Jim", "Anita"],
  pad = 30,
  thickness = 15
)

# Creates the Sankey diagram object for hiring option 1
data1 = go.Sankey(link = link, node = node1)

# Creates the figure for hiring option 1
fig1 = go.Figure(data=[data1])
fig1.update_layout(title_text='Hiring Solution 1 - Positions and Candidates',
                  font_size = 15)

# Plots the figure for hiring option 1
fig1.show()

# Defines the node labels for hiring option 2
node2 = dict(
  label = ["Python Programmer", "AI Engineer", "Web Designer","Database Administrator","Systems Engineer",
           "Ciara", "Jane", "Jim", "Juan"],
  pad = 30,
  thickness = 15 
)

# Creates the Sankey diagram object for hiring option 2
data2 = go.Sankey(link = link, node = node2)

# Creates the figure for hiring option 2
fig2 = go.Figure(data=[data2])
fig2.update_layout(title_text="Hiring Solution 2 - Positions and Candidates",
                  font_size = 15)

# Plots the figure for hiring option 2
fig2.show()

# GUI for Scenario 1 - Visualization Only

In [187]:
import tkinter as tk
from tkinter import ttk

# Creates a Tkinter window
window = tk.Tk()
window.geometry("600x550") 
window.title("Hiring Options")

# Create a label for the dropdown menu
label = tk.Label(window, text="Select a candidate to show if they fit the hiring conditions:", 
                 font=("Calibri", 14), fg="black")
label.pack(padx=10, pady=10)


# Create a dropdown menu with the candidate names
candidates = list(hires.keys())
candidate_var = tk.StringVar()
dropdown = ttk.Combobox(window, textvariable=candidate_var, values=candidates, state="readonly")

dropdown.pack()

# Create a text box to display the solutions
text = tk.Text(window, width=70, height=600, font=("Calibri", 12))
text.pack(padx=5, pady=50)

# Defines a function to update the text box based on the selected candidate
def update_text(event):
    # Clear the text box
    text.delete(1.0, tk.END)
 
    # Chooses candidate
    candidate = candidate_var.get()
    
    # Keeps track if the candidate was found
    found = False
    # Loops through the solutions and finds the ones that include the candidate
    for i, solution in enumerate(unique_solutions1):
        if candidate in solution.values():
            # Writes the solution number and the positions
            text.insert(tk.END, f"Solution {i+1}:\n")
            for position, hire in solution.items():
                    text.insert(tk.END, f"Position {position}: {hire}\n")
            text.insert(tk.END, "\n")
            found = True
    # Checks if found is still False
    if not found:
        # Print the message that there is no solution
        text.insert(tk.END, "There is no hiring solution that respects the conditions and includes this candidate." +
                   "\nPlease, try another one.")

            

# Bind the dropdown menu to the update function
dropdown.bind("<<ComboboxSelected>>", update_text)

# Start the main event loop
window.mainloop()

# Scenario 2 - Visualizations and GUI

In [82]:
df_scenario2 = pd.DataFrame(unique_solutions2)

df_scenario2.index.name = "Hiring option"
df_scenario2.reset_index(inplace=True)

df_scenario2.set_index("Hiring option", inplace=True)

df_scenario2.columns.name = df_scenario2.index.name
df_scenario2.index.name = None
df_scenario2.index = range(1, len(df_scenario2) + 1)


df_scenario2

Hiring option,AI_1,AI_2,AI_3,Python_1,Python_2,Database,Systems,Web
1,Anita,Jim,Juan,Bruce,Ciara,Jane,Bruce,Juan
2,Anita,Jim,Juan,Bruce,Ciara,Jane,Bruce,Anita
3,Anita,Jim,Juan,Bruce,Ciara,Jane,Jim,Juan
4,Anita,Jim,Juan,Bruce,Ciara,Jane,Jim,Anita
5,Anita,Jim,Juan,Bruce,Jane,Jane,Bruce,Juan
...,...,...,...,...,...,...,...,...
73,Jim,Juan,Peter,Ciara,Peter,Jane,Mary,Mary
74,Jim,Juan,Peter,Ciara,Peter,Jane,Mary,Juan
75,Jim,Juan,Peter,Ciara,Peter,Jane,Jim,Anita
76,Jim,Juan,Peter,Ciara,Peter,Jane,Jim,Mary


## Scenario 2 - Candidate x Position - Sankey Diagram

In [104]:
import plotly.graph_objects as go

labels = ["Ciara", "Peter", "Juan", "Jim", "Jane", "Mary","Bruce", "Anita", 
          "Python 1", "Python 2", "AI 1", "AI 2", "AI 3", "Web", "Systems", "Database"]

label_to_index = {label: index for index, label in enumerate(labels)}

sources = ["Ciara", "Ciara", "Peter","Peter", "Peter", "Peter", "Peter", "Juan", "Juan", "Juan", "Juan", 
           "Jim", "Jim", "Jim", "Jim","Jane", "Jane", "Jane", "Mary", "Mary", "Bruce", "Bruce", "Bruce", 
           "Anita","Anita", "Anita", "Anita"]
targets = ["Python 1", "Python 2", "Python 1", "Python 2", "AI 1", "AI 2", "AI 3", "Web", "AI 1", "AI 2", "AI 3", 
           "AI 1", "AI 2", "AI 3", "Systems", "Python 1", "Python 2", "Database", "Web", "Systems", "Systems", 
           "Python 1","Python 2", "Web","AI 1", "AI 2", "AI 3"]


source_indices = [label_to_index[source] for source in sources]
target_indices = [label_to_index[target] for target in targets]

# Define the source-target pairs
link = dict(
  arrowlen=15,
  source = source_indices,
  target = target_indices,
  value = [1]*len(source_indices),
)

# Define the node labels
node = dict(
  label = ["Ciara", "Peter", "Juan", "Jim", "Jane", "Mary", "Bruce", "Anita", "Python 1", "Python 2", 
           "AI 1", "AI 2", "AI 3", "Web", "Systems", "Database"],
  pad = 20, 
  thickness = 25
)

# Create the Sankey diagram object
data = go.Sankey(link = link, node = node)


fig = go.Figure(data=[data])

fig.update_layout(
    title={
        'text': "Positions candidates are qualified to occupy",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    font_size = 15
)

# Plot the figure
fig.show()

## Table visualization

In [194]:
from IPython.display import display, HTML


# Define the positions and the hires with their qualifications
positions2 = ["Python 1", "Python 2", "AI 1", "AI 2", "AI 3", "Web", "Database", "Systems"]

hires2 = {
    "Ciara": ["Python 1", "Python 2"],
    "Peter": ["Python 1", "Python 2", "AI 1", "AI 2", "AI 3"],
    "Juan": ["Web", "AI 1", "AI 2", "AI 3"],
    "Jim": ["AI 1", "AI 2", "AI 3", "Systems"],
    "Jane": ["Python 1", "Python 2", "Database"],
    "Mary": ["Web", "Systems"],
    "Bruce": ["Systems", "Python 1", "Python 2"],
    "Anita": ["Web", "AI 1", "AI 2", "AI 3"]
}

# Create a DataFrame with the available positions
df_positions2 = pd.DataFrame(positions2, columns=["Available Positions"])

# Convert the DataFrame to HTML
html_positions2 = df_positions2.to_html(index=False)

# DataFrame for each person, stored in a list
dfs2 = [pd.DataFrame({person: positions2}) for person, positions2 in hires.items()]

# Converts each DataFrame to HTML
html_strs = [df.to_html(index=False) for df in dfs2]

# Combines the HTML strings and display them


# Combine the HTML strings and display them
html_str = "<div style='text-align:center;'>" + \
            "<h2>Available Positions</h2>" + \
            "<div style='display:flex; flex-direction:row; justify-content:center;'>" + \
            f"<div style='padding:10px;'>{html_positions2}</div>" + \
            "</div></div>"

html_str += "<div style='text-align:center;'>" + \
           "<h2>Positions candidates are qualified to occupy</h2>" + \
           "<div style='display:flex; flex-direction:row; justify-content:center;'>" + \
           ''.join([f"<div style='padding:10px;'>{table}</div>" for table in html_strs]) + \
           "</div></div>"


display(HTML(html_str))

Available Positions
Python 1
Python 2
AI 1
AI 2
AI 3
Web
Database
Systems
Ciara
Python 1


## Frequency of each candidate as a hiring option

In [105]:
df_scenario2_frequency = pd.DataFrame(person_counts.items(), columns=["Hire Prospect", "Appears in solution (Times)"])
df_scenario2_frequency.sort_values(by="Appears in solution (Times)", ascending=False)

df_scenario2_frequency = df_scenario2_frequency.set_index("Hire Prospect")
df_scenario2_frequency.columns.name = df_scenario2_frequency.index.name
df_scenario2_frequency.index.name = None

df_scenario2_frequency

Hire Prospect,Appears in solution (Times)
Mary,22
Bruce,31
Ciara,48
Anita,56
Jim,56
Peter,61
Juan,74
Jane,77


## Graph visualization

In [115]:
import plotly.express as px

df = pd.DataFrame(person_counts.items(), columns=["Hire Prospect", "Appears in solution (Times)"])

fig = px.bar(df, x='Hire Prospect', y='Appears in solution (Times)', color='Hire Prospect',
             title='Number of times each candidate appears in the solutions',
             labels={'Appears in solution (Times)': 'Appears in solution (Times)',
                     'Hire Prospect': 'Hire Prospect'})

fig.update_layout(title_font=dict(size=20, color='black', family="Courier New, monospace"),
                 showlegend=False)

fig.show()


# GUI for Scenario 2 - Visualization Only

In [161]:
window = tk.Tk()

# Gets the screen width and height
screen_width = window.winfo_screenwidth()
screen_height = window.winfo_screenheight()

# Window size
window_width = 700
window_height = 650

# Calculate the position to start the window at
start_x = (screen_width // 2) - (window_width // 2)
start_y = 0

# Set the window size and start position
window.geometry(f"{window_width}x{window_height}+{start_x}+{start_y}")

window.title("Hiring Options")

# Label for the dropdown menu
label = tk.Label(window, text="Select a candidate to show the solutions:", 
                 font=("Calibri", 14), 
                 fg="black")

label.pack(padx=10, pady=10)

# Dropdown menu with the candidate names
candidates = list(hires.keys())
candidate_var = tk.StringVar()
dropdown = ttk.Combobox(window, 
                        textvariable=candidate_var, 
                        values=candidates, 
                        state="readonly")

dropdown.pack()

# Label candidate hiring option solutions
count_label = tk.Label(window, 
                       text="", 
                       font=("Calibri", 14), 
                       fg="black")

count_label.pack(padx=10, pady=10)


# Text box to display the solutions
text = tk.Text(window, 
               width=80, 
               height=50, 
               font=("Calibri", 12))

text.pack(padx=5, pady=50)

# Function to update the text box based on the selected candidate
def update_text(event):
    global count_label
    
    # Clears the text box
    text.delete(1.0, tk.END)
    
    # Gets the selected candidate
    candidate = candidate_var.get()
    
    # Updates the count label
    count = person_counts[candidate]
    count_label.config(text=f"{candidate} appears in {count} possible hiring options.")
    
    # Find the solutions that include the candidate
    for i, solution in enumerate(unique_solutions2):
        if candidate in solution.values():
            text.insert(tk.END, f"Solution {i+1}:\n")
            for position, hire in solution.items():
                text.insert(tk.END, f"Position {position}: {hire}\n")
            text.insert(tk.END, "\n")

# Binds the dropdown menu to the update function
dropdown.bind("<<ComboboxSelected>>", update_text)

window.mainloop()

# GUI for Alternative Hiring Solutions

In [172]:
from tkinter import messagebox

def solve_problem(num_hires):
    positions = ["Python_1", "Python_2", "AI_1", "AI_2", "Web", "Database", "Systems"]
    hires = {
        "Ciara": ["Python_1"],
        "Peter": ["Python_2", "AI_1", "AI_2"],
        "Juan": ["Web", "AI_1", "AI_2"],
        "Jim": ["AI_1", "AI_2", "Systems"],
        "Jane": ["Python_2", "Database"],
        "Mary": ["Web", "Systems"],
        "Bruce": ["Systems", "Python_2"],
        "Anita": ["Web", "AI_1", "AI_2"]
    }

    problem = Problem()

    for position in positions:
        problem.addVariable(position, list(hires.keys()))

    def ciara_python_constraint(python_1):
        return python_1 == "Ciara"

    def ai_constraint(ai_1, ai_2):
        return ai_1 != ai_2

    def db_constraint(database):
        return database == "Jane"

    def ciara_hiring_constraint(*positions):
        return len(set(positions)) <= num_hires + 1

    def max_two_positions_constraint(*positions):
        hires_count = dict()
        for hire in positions:
            if hire in hires_count:
                hires_count[hire] += 1
            else:
                hires_count[hire] = 1
        return all(count <= 2 for count in hires_count.values())

    def qualification_constraint(*args):
        for position, hire in zip(positions, args):
            if position not in hires[hire]:
                return False
        return True

    problem.addConstraint(max_two_positions_constraint, positions)
    problem.addConstraint(qualification_constraint, positions)
    problem.addConstraint(ciara_python_constraint, ["Python_1"])
    problem.addConstraint(ai_constraint, ["AI_1", "AI_2"])
    problem.addConstraint(db_constraint, ["Database"])
    problem.addConstraint(ciara_hiring_constraint, positions)

    solutions = problem.getSolutions()

    unique_solutions = []
    for solution in solutions:
        ai_positions = sorted([solution["AI_1"], solution["AI_2"]])
        solution["AI_1"], solution["AI_2"] = ai_positions[0], ai_positions[1]

        if solution not in unique_solutions:
            unique_solutions.append(solution)

    return unique_solutions

class AlternativeHiring(tk.Tk):
    def __init__(self):
        super().__init__()

        # Get the screen width and height
        screen_width = self.winfo_screenwidth()
        screen_height = self.winfo_screenheight()

        # Specify the window size
        window_width = 700
        window_height = 650

        # Calculate the position to start the window at
        start_x = (screen_width // 2) - (window_width // 2)
        start_y = 0

        # Set the window size and start position
        self.geometry(f"{window_width}x{window_height}+{start_x}+{start_y}")

        self.title("Alternative Hiring Solutions")

        self.label = Label(self, text="Enter an alternative amount of hires:",
                           font=("Calibri", 15),
                           fg="black")
        
        self.label.pack(pady=10)

        self.entry = Entry(self)
        self.entry.pack(pady=10)

        self.button = Button(self, text="Show Solutions", command=self.show_solutions)
        self.button.pack(pady=10)
        
        self.solutions_label = tk.Label(self, text="", font=("Calibri", 14), fg="black")
        self.solutions_label.pack(padx=10, pady=10)

        self.result_text = Text(self, wrap=tk.WORD, height=35, width=70)
        self.result_text.pack(padx =50, pady=10)

    def show_solutions(self):
        num_hires = self.entry.get()

        try:
            num_hires = int(num_hires)
            if 3 <= num_hires <= 7:
                solutions = solve_problem(num_hires)
                self.display_solutions(solutions)
                self.solutions_label.config(text=f"{len(solutions)} solutions found for this amount of hires.")
            elif num_hires < 3:
                messagebox.showerror("Error", "Ciara needs to hire at least 3 people to fill all positions.")
            else:
                messagebox.showerror("Error", "There are only 7 candidates available for hiring.")
        except ValueError:
            messagebox.showerror("Error", "Please enter a valid number.")

    def display_solutions(self, solutions):
        self.result_text.delete(1.0, tk.END)
        for i, solution in enumerate(solutions):
            self.result_text.insert(tk.END, f"Solution {i + 1}:\n")
            for position, hire in solution.items():
                self.result_text.insert(tk.END, f"Position {position}: {hire}\n")
            self.result_text.insert(tk.END, "\n")


if __name__ == "__main__":
    app = AlternativeHiring()
    app.mainloop()